In [ ]:
import os

# ==============================
# CONFIGURATION
# ==============================
BASE_DIR = "/scratch/hjh7hp/Watershed_22_2025_fall/Watershed22_with_new_summer/Sharadha_khola_watershed/1976_SA_1/salyan/model"
RHESSYS_EXE = "./RHESSysEastCoast/rhessysEC.7.2-2025-08-12-15-50-33"
WORLDHDR_DIR = "worldfiles"
OUTPUT_DIR = "SA_output_basin"
JOB_DIR = os.path.join(BASE_DIR, "SA_job")

SLURM_TIME = "10:00:00"
START_DATE = (1975, 1, 1, 1)
END_DATE = (1977, 12, 30, 1)

# --- Parameter Ranges (all set to 1.0) ---
PARAM_RANGES = {
    'gw': [1.0, 1.0, 1.0],
    's': [1.0, 1.0, 1.0],
    'sv': [1.0, 1.0],
    'svalt': [1.0, 1.0]
}



def main():
    os.makedirs(JOB_DIR, exist_ok=True)
    os.makedirs(os.path.join(BASE_DIR, OUTPUT_DIR), exist_ok=True)
    st_year, st_month, st_day, st_hour = START_DATE
    ed_year, ed_month, ed_day, ed_hour = END_DATE

    for i in range(1, NUM_DEFS_FOLDERS + 1):
        # Use fixed value 1.0 for all parameters
        gw_vals = PARAM_RANGES['gw']
        s_vals = PARAM_RANGES['s']
        sv_vals = PARAM_RANGES['sv']
        svalt_vals = PARAM_RANGES['svalt']

        param_str = (
            f"-gw {' '.join(map(str, gw_vals))} "
            f"-s {' '.join(map(str, s_vals))} "
            f"-sv {' '.join(map(str, sv_vals))} "
            f"-svalt {' '.join(map(str, svalt_vals))}"
        )

        worldfile_hdr = f"{WORLDHDR_DIR}/worldfile{i}.hdr"
        output_prefix = f"{OUTPUT_DIR}/Salyan{i}"
        job_path = os.path.join(JOB_DIR, f"basin{i}.job")
        jobname = f"basin{i}"
        job_content = f"""#!/bin/bash
#SBATCH --partition=standard
#SBATCH --ntasks=1
#SBATCH --mem=90gb
#SBATCH -t {SLURM_TIME}
#SBATCH --output=/dev/null
#SBATCH --error=basin_rosi.err
#SBATCH --job-name={jobname}
#SBATCH -A lband_group
#SBATCH --mail-type=ALL
cd {BASE_DIR}
{RHESSYS_EXE} -st {st_year} {st_month} {st_day} {st_hour} -ed {ed_year} {ed_month} {ed_day} {ed_hour} -b -g -dynRtZoff -BGC_flag -fracDirectNdep 0.5 -gwtoriparian -t tecfiles/tec_daily_cali.txt -w worldfiles/worldfile -whdr {worldfile_hdr} -r flows/subflow.txt flows/surfflow.txt -pre {output_prefix} {param_str}
"""
        with open(job_path, "w") as f:
            f.write(job_content)
    print(f"Created {NUM_DEFS_FOLDERS} job scripts in {JOB_DIR}")

if __name__ == "__main__":
    main()
